In [1]:
# Cell 1: Imports
from garminconnect import Garmin
from datetime import date
import json
import pandas as pd
import os
from getpass import getpass

# Cell 2: Login and setup (demo.py style)
def init_api():
    """Initialize Garmin API (demo.py style)"""
    tokenstore = os.path.expanduser("~/.garminconnect")
    
    try:
        print(f"Attempting to login using stored tokens from: {tokenstore}")
        api = Garmin()
        api.login(tokenstore)
        print("✅ Successfully logged in using stored tokens!")
        return api
    except Exception:
        print("No valid tokens found. Requesting fresh login credentials.")
    
    while True:
        try:
            email = input("Email address: ").strip()
            password = getpass("Password: ")
            
            print("Logging in with credentials...")
            api = Garmin(email=email, password=password, is_cn=False, return_on_mfa=True)
            result1, result2 = api.login()
            
            if result1 == "needs_mfa":
                print("Multi-factor authentication required")
                mfa_code = input("MFA one-time code: ")
                api.resume_login(result2, mfa_code)
                print("✅ MFA authentication successful!")
            
            api.garth.dump(tokenstore)
            print(f"✅ Login successful! Tokens saved to: {tokenstore}")
            return api
            
        except Exception as e:
            print(f"❌ Authentication failed: {e}")
            print("Please try again.")
            continue

api = init_api()
today = date.today()
print(f"Connected! Today's date: {today}")

Attempting to login using stored tokens from: C:\Users\Svitlana/.garminconnect
✅ Successfully logged in using stored tokens!
Connected! Today's date: 2025-10-24


In [8]:
# Cell: View Gear & Equipment Data
import json

print("🎽 GEAR & EQUIPMENT DATA")
print("=" * 60)

try:
    # First, get your user profile number (needed for gear API)
    print("\n1️⃣ Getting user info...")
    device_info = api.get_device_last_used()
    user_profile_number = device_info.get("userProfileNumber")
    print(f"   User Profile Number: {user_profile_number}")
    
    # Get all your gear
    print("\n2️⃣ Getting gear list...")
    gear_list = api.get_gear(user_profile_number)
    
    if not gear_list:
        print("   ⚠️ No gear registered in your account")
    else:
        print(f"   ✅ Found {len(gear_list)} gear item(s)\n")
        print("=" * 60)
        
        # Display each gear item
        for i, gear in enumerate(gear_list, 1):
            print(f"\n🔹 GEAR #{i}: {gear.get('displayName', 'Unnamed')}")
            print("-" * 60)
            
            # Basic info
            print(f"   Type: {gear.get('gearTypeName', 'Unknown')}")
            print(f"   Model: {gear.get('customMakeModel', 'Not specified')}")
            print(f"   UUID: {gear.get('uuid', 'N/A')}")
            print(f"   Status: {'Active' if gear.get('gearStatusName') == 'active' else 'Inactive'}")
            
            # Try to get usage statistics
            try:
                print(f"\n   📊 Usage Statistics:")
                gear_uuid = gear.get('uuid')
                if gear_uuid:
                    stats = api.get_gear_stats(gear_uuid)
                    
                    total_distance = stats.get('totalDistance', 0)
                    total_activities = stats.get('totalActivities', 0)
                    
                    print(f"      • Total Distance: {total_distance/1000:.2f} km")
                    print(f"      • Total Activities: {total_activities}")
                    print(f"      • Total Time: {stats.get('totalDuration', 0)/3600:.1f} hours")
                else:
                    print(f"      ⚠️ No UUID available")
            except Exception as e:
                print(f"      ⚠️ Stats not available: {e}")
            
            print()
    
    # Get gear defaults
    print("\n3️⃣ Getting gear defaults...")
    print("=" * 60)
    try:
        gear_defaults = api.get_gear_defaults(user_profile_number)
        
        if gear_defaults:
            print("\n📌 Default Gear Settings:")
            print(json.dumps(gear_defaults, indent=2))
        else:
            print("   ⚠️ No default gear settings")
    except Exception as e:
        print(f"   ⚠️ Could not get defaults: {e}")
    
    # Show raw gear data
    print("\n\n4️⃣ Raw Gear Data (all fields):")
    print("=" * 60)
    print(json.dumps(gear_list, indent=2))
    
except Exception as e:
    print(f"\n❌ Error getting gear data: {e}")
    print("\nTroubleshooting:")
    print("  • Make sure you're logged in")
    print("  • Check if you have any gear registered in Garmin Connect")

print("\n" + "=" * 60)
print("✨ Done!")

🎽 GEAR & EQUIPMENT DATA

1️⃣ Getting user info...
   User Profile Number: 89635332

2️⃣ Getting gear list...
   ✅ Found 8 gear item(s)


🔹 GEAR #1: 
------------------------------------------------------------
   Type: Shoes
   Model: Nike Zoom
   UUID: 54984ce40c6a4425a12f5a9a951c4414
   Status: Inactive

   📊 Usage Statistics:
      • Total Distance: 793.38 km
      • Total Activities: 106
      • Total Time: 0.0 hours


🔹 GEAR #2: None
------------------------------------------------------------
   Type: Shoes
   Model: Nike Vomero 16
   UUID: acf2f54d7ef342cf8a7fc113ecd4ead0
   Status: Inactive

   📊 Usage Statistics:
      • Total Distance: 1253.06 km
      • Total Activities: 129
      • Total Time: 0.0 hours


🔹 GEAR #3: None
------------------------------------------------------------
   Type: Shoes
   Model: Asics Gel Nimbus 25
   UUID: 5d8f33f3f3624cc68a2469dd38c96bf7
   Status: Inactive

   📊 Usage Statistics:
      • Total Distance: 909.97 km
      • Total Activities: 86
  

In [9]:
# Cell: See gear stats for Asics Megablast
import json

gear_uuid = "ad3545999f2f46558c214d909f4ec0fb"  # Asics Megablast

print("📊 GEAR STATS - Asics Megablast")
print("=" * 60)

stats = api.get_gear_stats(gear_uuid)

print(json.dumps(stats, indent=2))

📊 GEAR STATS - Asics Megablast
{
  "gearPk": 45528920,
  "uuid": "ad3545999f2f46558c214d909f4ec0fb",
  "createDate": 1757610535000,
  "updateDate": 1761236872000,
  "totalDistance": 57608.08056640625,
  "totalActivities": 5,
  "isProcessing": false,
  "processing": false
}


In [11]:
# Cell: Get basic gear info for Megablast
import json

# Get all gear
device_info = api.get_device_last_used()
user_profile_number = device_info.get("userProfileNumber")
gear_list = api.get_gear(user_profile_number)

# Find Megablast
gear_uuid = "ad3545999f2f46558c214d909f4ec0fb"
megablast = next((g for g in gear_list if g['uuid'] == gear_uuid), None)

print("📊 BASIC GEAR INFO - Asics Megablast")
print("=" * 60)

if megablast:
    print(json.dumps(megablast, indent=2))
else:
    print("Not found")

📊 BASIC GEAR INFO - Asics Megablast
{
  "gearPk": 45528920,
  "uuid": "ad3545999f2f46558c214d909f4ec0fb",
  "userProfilePk": 89635332,
  "gearMakeName": "Other",
  "gearModelName": "Unknown Shoes",
  "gearTypeName": "Shoes",
  "gearStatusName": "active",
  "displayName": null,
  "customMakeModel": "Asics Megablast",
  "imageNameLarge": null,
  "imageNameMedium": null,
  "imageNameSmall": null,
  "dateBegin": "2025-09-11T00:00:00.0",
  "dateEnd": null,
  "maximumMeters": 1000000.0,
  "notified": false,
  "createDate": "2025-09-11T17:08:55.0",
  "updateDate": "2025-09-11T17:08:55.0"
}


In [12]:
# Cell: Compare basic info vs stats for Megablast
import json

gear_uuid = "ad3545999f2f46558c214d909f4ec0fb"

# Get basic info (from gear list)
device_info = api.get_device_last_used()
user_profile_number = device_info.get("userProfileNumber")
gear_list = api.get_gear(user_profile_number)
megablast_basic = next((g for g in gear_list if g['uuid'] == gear_uuid), None)

# Get stats
megablast_stats = api.get_gear_stats(gear_uuid)

print("📊 ASICS MEGABLAST - COMPARISON")
print("=" * 60)

print("\n1️⃣ BASIC INFO (from get_gear):")
print("-" * 60)
print(json.dumps(megablast_basic, indent=2))

print("\n\n2️⃣ USAGE STATS (from get_gear_stats):")
print("-" * 60)
print(json.dumps(megablast_stats, indent=2))

📊 ASICS MEGABLAST - COMPARISON

1️⃣ BASIC INFO (from get_gear):
------------------------------------------------------------
{
  "gearPk": 45528920,
  "uuid": "ad3545999f2f46558c214d909f4ec0fb",
  "userProfilePk": 89635332,
  "gearMakeName": "Other",
  "gearModelName": "Unknown Shoes",
  "gearTypeName": "Shoes",
  "gearStatusName": "active",
  "displayName": null,
  "customMakeModel": "Asics Megablast",
  "imageNameLarge": null,
  "imageNameMedium": null,
  "imageNameSmall": null,
  "dateBegin": "2025-09-11T00:00:00.0",
  "dateEnd": null,
  "maximumMeters": 1000000.0,
  "notified": false,
  "createDate": "2025-09-11T17:08:55.0",
  "updateDate": "2025-09-11T17:08:55.0"
}


2️⃣ USAGE STATS (from get_gear_stats):
------------------------------------------------------------
{
  "gearPk": 45528920,
  "uuid": "ad3545999f2f46558c214d909f4ec0fb",
  "createDate": 1757610535000,
  "updateDate": 1761236872000,
  "totalDistance": 57608.08056640625,
  "totalActivities": 5,
  "isProcessing": false,


In [2]:
# Cell: Load gear data to database
import sqlite3
import pandas as pd

# 1. Get the data (you already know how to do this)
device_info = api.get_device_last_used()
user_profile_number = device_info.get("userProfileNumber")
gear_list = api.get_gear(user_profile_number)

# 2. Convert to pandas DataFrame
df = pd.DataFrame(gear_list)

# 3. Save to your database
db_path = r'C:\Users\Svitlana\OneDrive\Garmin\garmin.db'
conn = sqlite3.connect(db_path)
df.to_sql('raw_gear_list', conn, if_exists='replace', index=False)
conn.close()

print("✅ Done! Your data is now in the database.")

✅ Done! Your data is now in the database.
